In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
%matplotlib inline

In [3]:
customer = pd.read_csv('../Customer.csv')
trade = pd.read_csv('../Trade.csv')

In [4]:

ProfileReport(trade.sample(100000))

Number of variables,9
Number of observations,100000
Total Missing (%),7.6%
Total size in memory,6.9 MiB
Average record size in memory,72.0 B
Numeric,6
Categorical,2
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [12]:
trade.groupby('CustomerIdx')['IsinIdx'].count().describe()

count      3439.000000
mean       1966.275371
std        7911.310868
min           1.000000
25%           7.000000
50%          55.000000
75%         574.000000
max      154077.000000
Name: IsinIdx, dtype: float64

In [19]:
mm = pd.read_csv('../MarketData_Macro.csv').head(10)

In [20]:
mm.columns[:100]

Index(['DateKey', 'SSE', 'DAX', 'EUROSTOXX', 'VSTOXX', 'FTSE100', 'HSI',
       'NIKKEI', 'DOWJONES_INDU', 'SP500', 'VIX', 'FX_USD.ARS', 'FX_USD.AUD',
       'FX_USD.BRL', 'FX_USD.CAD', 'FX_USD.CHF', 'FX_USD.CNO', 'FX_USD.CNY',
       'FX_USD.EUR', 'FX_USD.GBP', 'FX_USD.HKD', 'FX_USD.IDR', 'FX_USD.JPY',
       'FX_USD.NOK', 'FX_USD.SGD', 'FX_USD.TRY', 'FX_USD.ZAR',
       'MoneyMarket_ARS3M', 'MoneyMarket_AUD3M', 'MoneyMarket_CAD3M',
       'MoneyMarket_CHF3M', 'MoneyMarket_CNO3M', 'MoneyMarket_CNY3M',
       'MoneyMarket_EUR3M', 'MoneyMarket_GBP3M', 'MoneyMarket_HKD3M',
       'MoneyMarket_IDR3M', 'MoneyMarket_JPY3M', 'MoneyMarket_NOK3M',
       'MoneyMarket_SGD3M', 'MoneyMarket_TRY3M', 'MoneyMarket_USD3M',
       'MoneyMarket_ZAR3M', 'Swap_ARS10Y', 'Swap_ARS2Y', 'Swap_ARS5Y',
       'Swap_AUD10Y', 'Swap_AUD2Y', 'Swap_AUD30Y', 'Swap_AUD5Y', 'Swap_BRL10Y',
       'Swap_BRL2Y', 'Swap_BRL5Y', 'Swap_CAD10Y', 'Swap_CAD2Y', 'Swap_CAD30Y',
       'Swap_CAD5Y', 'Swap_CHF10Y', 'Swap_CHF2Y', 'S

In [15]:
pd.read_csv('../Market.csv', nrows=15)

,IsinIdx,DateKey,Price,Yield,ZSpread
0,1,20160101,104.250,7.835,5.505
1,7,20160101,107.500,7.520,5.541
2,102,20160101,100.746,4.048,2.085
3,331,20160101,112.790,-0.752,-0.215
4,345,20160101,113.383,-0.667,-0.272
5,531,20160101,113.575,-0.513,-0.226
6,1251,20160101,108.891,0.238,-0.380
7,1252,20160101,108.937,0.271,-0.380
8,1253,20160101,112.782,0.298,-0.381
9,1254,20160101,110.945,0.377,-0.372
